# 간단한 전이학습 시켜보기

1. 주제를 정한다.
2. 주제에 맞는 태스크를 판단한다.
3. 해당 태스크를 수행하는 모델을 huggingface에서 찾는다.
4. 적절한 데이터셋을 정제하여 준비한다.
5. 3에서 찾은 모델을 로드하여 '간단하게' 전이학습을 진행한다.
6. 추론 결과를 확인한다.

1. 주제 : 한국어 온라인 댓글 및 메시지에 숨겨진 비꼬는 뉘앙스 탐지 분류 모델 개발
2. task : 문장을 입력받아, 그 문장이 '비꼼'인지 '진심'인지 둘 중 하나로 분류 (이진 텍트스 분류 1: 비꼼, 0: 진심)
3. hugging face에서 모델 찾기 : 한국어 텍스트 분류 태스크에 적합한 사전 학습된 모델 탐색 -> klue/bert-base or koelectra
4. 데이터셋 준비 : 데이터 수집..... (비꼼 250개, 진심 250개) -> 깔끔한 텍스트 파일로 저장 (문장, 라벨 형식 / 비꼼=1, 진심=0)
5. 3번에서 찾은 모델에 4번에서 저장한 데이터셋을 학습시킴
6. 학습에 사용하지 않은 새로운 텍스트 문장을 입력해서 예측 확인

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

# 데이터 로드
df = pd.read_csv('sarcasm_data.csv')

# 학습 데이터, 테스트 데이터 분리
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [2]:
# 데이터 HF Dataset으로 변환 (Trainer는 dataframe을 바로 사용할 수 없어서 dataset 객체로 바꿔주어야 함.)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 토크나이저
model_name = 'klue/bert-base'

tokenizer = AutoTokenizer.from_pretrained(model_name)

# 텍스트를 모델 입력용 숫자 시퀀스로 변환
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

# 원래의 dataset의 각 row에 새로운 컬럼인 input_ids와 attention_mask를 추가
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Map:   0%|          | 0/192 [00:00<?, ? examples/s]

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [ ]:
# 데이터셋의 특정 컬럼을 PyTorch tensor로 변환
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [5]:
# 모델 불러오기
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#!pip install transformers[torch]
#!pip install accelerate -U

In [ ]:
from transformers import Trainer, TrainingArguments

# TrainingArguments (학습 설정값들을 한 곳에 모아둔 객체)
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    report_to="none"
)

# Trainer (Hugging Face가 제공하는 훈련 루프 자동화 도구 -> Trainer가 자동으로 데이터 불러오기, forward, backward, evaluation 진행)
# 불러온 모델과 설정해둔 값들과 준비해둔 train,test dataset을 가지고 훈련 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Trainer 학습
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.646200,0.416853
2,0.362500,0.344558
3,0.269900,0.301364
4,0.122000,0.310624
5,0.055700,0.313760


TrainOutput(global_step=60, training_loss=0.2533198595046997, metrics={'train_runtime': 152.2143, 'train_samples_per_second': 6.307, 'train_steps_per_second': 0.394, 'total_flos': 252586613145600.0, 'train_loss': 0.2533198595046997, 'epoch': 5.0})

In [9]:
# 내가 학습시킨 모델 저장
save_directory = "./my_sarcasm_model"
trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

('./my_sarcasm_model/tokenizer_config.json',
 './my_sarcasm_model/special_tokens_map.json',
 './my_sarcasm_model/vocab.txt',
 './my_sarcasm_model/added_tokens.json',
 './my_sarcasm_model/tokenizer.json')

In [10]:
from transformers import pipeline

# 테스트해보기
sarcasm_clf = pipeline('text-classification', model='./my_sarcasm_model')

while True:
    input_sentence = input('문장을 입력하세요 (마치고 싶으면 종료를 입력하세요.)\n')

    if input_sentence != '종료':
        result = sarcasm_clf(input_sentence)[0]

        res = result['label']
        score = result['score']

        label = '비꼼' if res == 'LABEL_1' else '진심'

        print(f'문장:', input_sentence)
        print(f'예측 : {label} | 정확도 : {score:.4f}')
        print("-" * 50)

    else:
        break

Device set to use cuda:0


문장을 입력하세요 (마치고 싶으면 종료를 입력하세요.)
이야 너 정말 창의적이다.
문장: 이야 너 정말 창의적이다.
예측 : 비꼼 | 정확도 : 0.9099
--------------------------------------------------
문장을 입력하세요 (마치고 싶으면 종료를 입력하세요.)
자세한 설명 정말 감사드립니다.
문장: 자세한 설명 정말 감사드립니다.
예측 : 진심 | 정확도 : 0.9564
--------------------------------------------------
문장을 입력하세요 (마치고 싶으면 종료를 입력하세요.)
팀운 레전드
문장: 팀운 레전드
예측 : 비꼼 | 정확도 : 0.7559
--------------------------------------------------
문장을 입력하세요 (마치고 싶으면 종료를 입력하세요.)
덕분에 많이 배우고 갑니다.
문장: 덕분에 많이 배우고 갑니다.
예측 : 진심 | 정확도 : 0.9766
--------------------------------------------------
문장을 입력하세요 (마치고 싶으면 종료를 입력하세요.)
오랜만이네 연락, 참 빨리도 한다.
문장: 오랜만이네 연락, 참 빨리도 한다.
예측 : 비꼼 | 정확도 : 0.9269
--------------------------------------------------
문장을 입력하세요 (마치고 싶으면 종료를 입력하세요.)
잘~한다.
문장: 잘~한다.
예측 : 비꼼 | 정확도 : 0.8108
--------------------------------------------------
문장을 입력하세요 (마치고 싶으면 종료를 입력하세요.)
그렇게 하면 퍽이나 되겠다.
문장: 그렇게 하면 퍽이나 되겠다.
예측 : 비꼼 | 정확도 : 0.9653
--------------------------------------------------
문장을 입력하세요 (마치고 싶으면 종료를 입력하세요.)
너 웃는 모습만 봐

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


문장: 너 오늘 정말 예쁘다!
예측 : 진심 | 정확도 : 0.9250
--------------------------------------------------
문장을 입력하세요 (마치고 싶으면 종료를 입력하세요.)
종료
